In [1]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-4 

Cloning into 'ATML-PA-4'...


remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 173 (delta 114), reused 108 (delta 54), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 351.25 KiB | 7.17 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [122]:
!git pull  http://github.com/Zapy67/ATML-PA-4 atm-10

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 3.85 KiB | 1.28 MiB/s, done.
From http://github.com/Zapy67/ATML-PA-4
 * branch            atm-10     -> FETCH_HEAD
Updating 93b7aad..822c2c8
Fast-forward
 fed_lib/utils.py |   3 +-
 fedsam.ipynb     | 618 ++++++++++++++++++++++++++++++++++++++++++++++++-------
 2 files changed, 551 insertions(+), 70 deletions(-)


In [1]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [2]:
!ls

'ATML - PA 4.pdf'   fed_lib	   LICENSE     task1.ipynb   task3.ipynb
 data		    fedsam.ipynb   README.md   task2.ipynb   task4.ipynb


#  FedSAM on heterogeneous domains

In [3]:
import torch
import torch.nn as nn

In [4]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [6]:
!pip install fedlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:

In [131]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

<module 'fed_lib.fed_model' from '/kaggle/working/ATML-PA-4/fed_lib/fed_model.py'>

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
trainset, testset = utils.get_cifar10()

In [8]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [9]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.104 0.084 0.211 0.129 0.093 0.097 0.005 0.073 0.204]
Entropy of Label Distribution H(P(Y)): 1.943856784330996
Normalized Entropy of Label Distribution: 0.8846873480213884

Label Distribution (P(Y)): [0.084 0.14  0.055 0.147 0.079 0.08  0.161 0.089 0.055 0.109]
Entropy of Label Distribution H(P(Y)): 2.1443700223362328
Normalized Entropy of Label Distribution: 0.9312880678593787

Label Distribution (P(Y)): [0.027 0.155 0.168 0.059 0.195 0.06  0.093 0.125 0.117]
Entropy of Label Distribution H(P(Y)): 1.9894101534189823
Normalized Entropy of Label Distribution: 0.9054195797458361

Label Distribution (P(Y)): [0.12  0.057 0.017 0.125 0.011 0.182 0.055 0.153 0.067 0.214]
Entropy of Label Distribution H(P(Y)): 1.9727481056667706
Normalized Entropy of Label Distribution: 0.8567536164761715

Label Distribution (P(Y)): [0.181 0.05  0.027 0.022 0.122 0.085 0.202 0.059 0.042 0.209]
Entropy of Label Distribution H(P(Y)): 1.9669507151120387
Normalized Entropy of Label Di

In [10]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [11]:
client_weights

[0.21684, 0.22048, 0.2036, 0.19172, 0.16736]

In [156]:
partition = "dirichlet"
alpha = 1
num_clients = 1
domains=None
fed = fed_model.Federation(num_clients, None, partition, domains, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

In [147]:
client_loaders = fed.client_dataloaders
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]

In [168]:
fed.set_method(fed_methods.FedSAM(sam_rho=0, 
             num_local_steps=1563, 
             client_aggregation_weights= client_weights))

In [170]:
fed.train(5, lr=3e-4, verbose=False, momentum = 0, weight_decay=0)


--- Round 1/5 ---
Training Clients
Training Client 1/1
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: 2.3152, Test Acc: 10.64%

--- Round 2/5 ---
Training Clients
Training Client 1/1


Training Server
Evaluate on round 2:


FedSAM  | Test Loss: 2.3151, Test Acc: 10.43%

--- Round 3/5 ---
Training Clients
Training Client 1/1


Training Server
Evaluate on round 3:


...

Training Server
Evaluate on round 4:


FedSAM  | Test Loss: 2.3158, Test Acc: 10.29%

--- Round 5/5 ---
Training Clients
Training Client 1/1


Training Server
Evaluate on round 5:


FedSAM  | Test Loss: 2.3158, Test Acc: 10.34%
Training Complete!


FedSAM  | Test Loss: 2.3158, Test Acc: 10.34%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.0.weight                1.144042e+00 6.861756e-02
backbone.2.block.3.weight                1.091380e+00 6.684693e-02
backbone.1.block.3.weight                9.403731e-01 7.162795e-02
backbone.1.block.0.weight                8.609227e-01 6.557172e-02
backbone.0.block.3.weight                6.900007e-01 6.428969e-02
Overall Statistics:
Total L2 difference:        2.240082e+00
Total parameters:           288,746
